In [ ]:
from rmgpy.molecule import Molecule, Atom, getElement
from rdkit.Chem import AllChem, Pharm3D

In [2]:
import os
import re
import logging
import external.cclib as cclib
import time
from subprocess import Popen
from copy import deepcopy
import numpy
import shutil
import math
import sqlite3 as lite

import rmgpy
#from rmgpy.data.kinetics.transitionstates import TransitionStates
from rmgpy.molecule import Molecule, Atom, getElement
#import rmgpy.molecule.
from rmgpy.species import Species, TransitionState
from rmgpy.reaction import Reaction
from rmgpy.kinetics import Arrhenius, Eckart
from rmgpy.statmech import Conformer, IdealGasTranslation, NonlinearRotor, HarmonicOscillator, LinearRotor
from qm.transitionstates import DistanceData, TransitionStateDepository, TSGroups, TransitionStates
from rmgpy.cantherm.main import CanTherm
from rmgpy.cantherm.kinetics import KineticsJob

In [3]:
from autotst.molecule import *
import autotst.conformer
from autotst.reaction import *
from autotst.conformer import *
from rmgpy.reaction import Reaction

thermo.py:833 loadLibraries INFO Loading thermodynamics library from primaryThermoLibrary.py in /Users/nathan/Code/RMG-database/input/thermo/libraries...
thermo.py:833 loadLibraries INFO Loading thermodynamics library from KlippensteinH2O2.py in /Users/nathan/Code/RMG-database/input/thermo/libraries...
thermo.py:833 loadLibraries INFO Loading thermodynamics library from thermo_DFT_CCSDTF12_BAC.py in /Users/nathan/Code/RMG-database/input/thermo/libraries...
thermo.py:833 loadLibraries INFO Loading thermodynamics library from CBS_QB3_1dHR.py in /Users/nathan/Code/RMG-database/input/thermo/libraries...
thermo.py:850 loadGroups INFO Loading thermodynamics group database from /Users/nathan/Code/RMG-database/input/thermo/groups...
transport.py:294 loadGroups INFO Loading transport group database from /Users/nathan/Code/RMG-database/input/transport/groups...
database.py:172 loadFamilies INFO Loading the user-specified kinetics families from /Users/nathan/Code/RMG-database/input/kinetics/famil

In [4]:
from rmgpy.data.kinetics.common import KineticsError, saveEntry

In [5]:
a = AutoTST_Reaction("OO+[CH2]CCCC_[O]O+CCCCC", "H_Abstraction")
a

[Molecule(SMILES="OO"), Molecule(SMILES="[CH2]CCCC")]
[TemplateReaction(reactants=[Species(label="", molecule=[Molecule(SMILES="OO")]), Species(label="", molecule=[Molecule(SMILES="[CH2]CCCC")])], products=[Species(label="", molecule=[Molecule(SMILES="CCCCC")]), Species(label="", molecule=[Molecule(SMILES="[O]O")])], degeneracy=2.0, pairs=[[Species(label="", molecule=[Molecule(SMILES="OO")]), Species(label="", molecule=[Molecule(SMILES="[O]O")])], [Species(label="", molecule=[Molecule(SMILES="[CH2]CCCC")]), Species(label="", molecule=[Molecule(SMILES="CCCCC")])]], family='H_Abstraction', template=['OOH', 'Csj/Cs/H2'])]
OO + [CH2]CCCC <=> CCCCC + [O]O


Exception TypeError: "Argument 'other' has incorrect type (expected rmgpy.molecule.graph.Graph, got rmgpy.species.Species)" in 'rmgpy.reaction._isomorphicSpeciesList' ignored
Exception TypeError: "Argument 'other' has incorrect type (expected rmgpy.molecule.graph.Graph, got rmgpy.species.Species)" in 'rmgpy.reaction._isomorphicSpeciesList' ignored
Exception TypeError: "Argument 'other' has incorrect type (expected rmgpy.molecule.graph.Graph, got rmgpy.species.Species)" in 'rmgpy.reaction._isomorphicSpeciesList' ignored
Exception TypeError: "Argument 'other' has incorrect type (expected rmgpy.molecule.graph.Graph, got rmgpy.species.Species)" in 'rmgpy.reaction._isomorphicSpeciesList' ignored


ValueError: No atom in the molecule has the label "*1".

In [ ]:
a.ts.view_ts()

In [ ]:
rd_mol = react.toRDKitMol(removeHs=False)
rdkit.Chem.AllChem.EmbedMolecule(rd_mol)
a.ts.view_ts(rd_mol)

In [ ]:
r1, r2 = a.rmg_reaction.reactants
r1 = r1.toRDKitMol(removeHs=False)
r2 = r2.toRDKitMol(removeHs=False)

combo = Chem.CombineMols(r1,r2)
rdkit.Chem.AllChem.EmbedMolecule(combo)
combo

In [ ]:
a.ts.view_ts(combo)

In [ ]:
r = Reaction(reactants=[Molecule(SMILES="OO"), Molecule(SMILES="[CH2]CCC")], products=[Molecule(SMILES="[O]O"),Molecule(SMILES="CCCC")])

In [ ]:
reacts = rmg_database.kinetics.generateReactionsFromFamilies(r.reactants, r.products)
r1, r2 = reacts
r1

In [ ]:
a, b = r1.reactants
merge = Molecule.merge(a,b)
merge

In [ ]:
def view_ts(mol=None):
    """
    A method designed to create a 3D figure of the Multi_Molecule with py3Dmol
    """
    

    mb = Chem.MolToMolBlock(mol)
    p = py3Dmol.view(width=400, height=400)
    p.addModel(mb, "sdf")
    p.setStyle({'stick':{}})
    p.setBackgroundColor('0xeeeeee')
    p.zoomTo()
    return p.show()

In [ ]:
from autotst.reaction import *
rd_merge = merge.toRDKitMol(removeHs=False)
rdkit.Chem.AllChem.EmbedMolecule(rd_merge)
view_ts(rd_merge)


In [ ]:
bm = rdkit.Chem.rdDistGeom.GetMoleculeBoundsMatrix(rd_merge)
bm

In [ ]:
view_ts(rd_merge)

In [ ]:
for reaction in reacts:
    print reaction
    # Check if any of the RMG proposed reactions matches the reaction in the mechanism
    if r.isIsomorphic(reaction):
        atom_labels_reactants = dict([(lbl[0], False) for lbl in reaction.labeledAtoms])
        atom_labels_products = dict([(lbl[0], False) for lbl in reaction.labeledAtoms])

        for reactant in reaction.reactants:
            reactant.clearLabeledAtoms()
            for atom in reactant.atoms:
                for atom_label in reaction.labeledAtoms:
                    if atom == atom_label[1]:
                        atom.label = atom_label[0]
                        atom_labels_reactants[atom_label[0]] = True

        for product in reaction.products:
            product.clearLabeledAtoms()
            for atom in product.atoms:
                for atom_label in reaction.labeledAtoms:
                    if atom == atom_label[1]:
                        atom.label = atom_label[0]
                        atom_labels_products[atom_label[0]] = True

        if all(atom_labels_reactants.values()) and all(atom_labels_products.values()):
            # We successfully labeled all of the atoms
            break
reaction

In [ ]:
data = ts_database.groups.estimateDistancesUsingGroupAdditivity(r1)
data

In [ ]:
d12 = data.distances["d12"]
d13 = data.distances["d13"]
d23 = data.distances["d23"]

In [ ]:
lbl1 = merge.getLabeledAtom("*1").sortingLabel
lbl2 = merge.getLabeledAtom("*2").sortingLabel
lbl3 = merge.getLabeledAtom("*3").sortingLabel

In [ ]:
bm

In [ ]:
blem = Molecule.merge(reaction.reactants[0], reaction.reactants[1])
mol = blem.toRDKitMol(removeHs=False)
mol

In [ ]:
if reaction.family.lower() in ['h_abstraction', 'r_addition_multiplebond', 'intra_h_migration']:
    lbl1 = blem.getLabeledAtom('*1').sortingLabel
    lbl2 = blem.getLabeledAtom('*2').sortingLabel
    lbl3 = blem.getLabeledAtom('*3').sortingLabel
elif reaction.family.lower() in ['disproportionation']:
    lbl1 = blem.getLabeledAtom('*2').sortingLabel
    lbl2 = blem.getLabeledAtom('*4').sortingLabel
    lbl3 = blem.getLabeledAtom('*1').sortingLabel

labels = [lbl1, lbl2, lbl3]
atomMatch = ((lbl1,),(lbl2,),(lbl3,))

labels, atomMatch

In [ ]:
len(blem.atoms)

In [ ]:
def setLimits(bm, lbl1, lbl2, value, uncertainty):
    if lbl1 > lbl2:
        bm[lbl2][lbl1] = value + uncertainty/2
        bm[lbl1][lbl2] = max(0,value - uncertainty/2)
    else:
        bm[lbl2][lbl1] = max(0,value - uncertainty/2)
        bm[lbl1][lbl2] = value + uncertainty/2

    return bm

In [ ]:
bm = rdkit.Chem.rdDistGeom.GetMoleculeBoundsMatrix(mol)
bm

In [ ]:
bm.shape

In [ ]:
lbl1, lbl2, lbl3 = labels

uncertainties = {'d12':0.02, 'd13':0.02, 'd23':0.02 } # distanceData.uncertainties or {'d12':0.1, 'd13':0.1, 'd23':0.1 } # default if uncertainty is None
bm = setLimits(bm, lbl1, lbl2, data.distances['d12'], uncertainties['d12'])
bm = setLimits(bm, lbl2, lbl3, data.distances['d23'], uncertainties['d23'])
bm = setLimits(bm, lbl1, lbl3, data.distances['d13'], uncertainties['d13'])
bm

In [ ]:
def optimize( rdmol, boundsMatrix=None, atomMatch=None):
    """

    Optimizes the rdmol object using UFF.
    Determines the energy level for each of the conformers identified in rdmol.GetConformer.


    :param rdmol:
    :param boundsMatrix:
    :param atomMatch:
    :return rdmol, minEid (index of the lowest energy conformer)
    """

    energy=0.0
    minEid=0;
    lowestE=9.999999e99;#start with a very high number, which would never be reached
    crude = Chem.Mol(rdmol.ToBinary())

    for conf in rdmol.GetConformers():
        if boundsMatrix is None:
            AllChem.UFFOptimizeMolecule(rdmol,confId=conf.GetId())
            energy=AllChem.UFFGetMoleculeForceField(rdmol,confId=conf.GetId()).CalcEnergy()
        else:
            eBefore, energy = Pharm3D.EmbedLib.OptimizeMol(rdmol, boundsMatrix, atomMatches=atomMatch, forceConstant=100000.0)

        if energy < lowestE:
            minEid = conf.GetId()
            lowestE = energy

    return rdmol, minEid

In [ ]:
def rd_embed(rdmol, numConfAttempts, bm=None, match=None):
    """
    Embed the RDKit molecule and create the crude molecule file.
    """
    if bm is None: #bm = bounds matrix?
        AllChem.EmbedMultipleConfs(rdmol, numConfAttempts ,randomSeed=1)
        crude = Chem.Mol(rdmol.ToBinary())
        rdmol, minEid = optimize(rdmol)
    else:
        """
        Embed the molecule according to the bounds matrix. Built to handle possible failures
        of some of the embedding attempts.
        """
        rdmol.RemoveAllConformers()
        for i in range(0,numConfAttempts):
            try:
                Pharm3D.EmbedLib.EmbedMol(rdmol, bm, atomMatch=match)
                break
            except ValueError:
                logging.info("RDKit failed to embed on attempt {0} of {1}".format(i + 1, numConfAttempts))
                # What to do next (what if they all fail?) !!!!!
            except RuntimeError:
                raise RDKitFailedError()
        else:
            logging.error("RDKit failed all attempts to embed")
            return None, None

        """
        RDKit currently embeds the conformers and sets the id as 0, so even though multiple
        conformers have been generated, only 1 can be called. Below the id's are resolved.
        """
        for i in range(len(rdmol.GetConformers())):
            rdmol.GetConformers()[i].SetId(i)

        crude = Chem.Mol(rdmol.ToBinary())
        rdmol, minEid = optimize(rdmol, boundsMatrix=bm, atomMatch=match)

    return rdmol, minEid

In [ ]:
from rdkit.Chem import AllChem, Pharm3D
new_mol, min_e = rd_embed(mol, 1000, bm, atomMatch)
new_mol

In [ ]:


mol_list = AllChem.MolToMolBlock(new_mol).split('\n')
ase_atoms = []
for i, line in enumerate(mol_list):

    if i > 3:

        try:
            atom0, atom1, bond, rest = line
            atom0 = int(atom0)
            atom0 = int(atom1)
            bond = float(bond)

        except ValueError:
            try:
                x, y, z, symbol = line.split()[0:4]
                x = float(x)
                y = float(y)
                z = float(z)
                # print symbol

                ase_atoms.append(Atom(symbol=symbol, position=(x, y, z)))

            except:
                continue

view_mol = Atoms(ase_atoms)
view_mol

In [ ]:
from ase.visualize import view
view(view_mol, viewer='x3d')

In [ ]:
ts = TransitionStates()
"""ts.groups = groups
ts.depository = training"""
ts.family = rmg_database.kinetics.families.values()[0]

ts.load(path, local_context, global_context)

In [ ]:
e = ts.groups.descendTree(r1.reactants[0], r1.reactants[0].getLabeledAtoms())
a = e.data
a.distances

In [ ]:
from rmgpy.species import Species, TransitionState

In [ ]:
from rmgpy.statmech import Conformer, IdealGasTranslation, NonlinearRotor, HarmonicOscillator, LinearRotor

In [ ]:

from rmgpy.data.base import *

In [ ]:
import os
import re
import logging
import external.cclib as cclib
import time
from subprocess import Popen
from copy import deepcopy
import numpy
import shutil
import math
import sqlite3 as lite

import rmgpy
from rmgpy.data.kinetics.transitionstates import TransitionStates
from rmgpy.molecule import Molecule, Atom, getElement
from rmgpy.species import Species, TransitionState
from rmgpy.reaction import Reaction
from rmgpy.kinetics import Arrhenius, Eckart
from rmgpy.statmech import Conformer, IdealGasTranslation, NonlinearRotor, HarmonicOscillator, LinearRotor

from qm.transitionstates import DistanceData, TransitionStateDepository, TSGroups, TransitionStates
from rmgpy.cantherm.main import CanTherm
from rmgpy.cantherm.kinetics import KineticsJob


In [ ]:
family = "H_Abstraction"

global_context = { '__builtins__': None }
local_context={'DistanceData': DistanceData}
path = "database/" + family

fpath = os.path.join(path,'TS_training', 'reactions.py')

training = TransitionStateDepository(label='H_Abstraction/TS_training')
training.load(fpath, local_context, global_context )

groups = TSGroups()
groups.load(os.path.join(path, "TS_groups.py"), local_context, global_context)
groups.label = family
groups.entries

In [ ]:
from rmgpy.data.rmg import RMGDatabase
rxnFamilies = ["H_Abstraction"]
rmgDatabase = RMGDatabase()
databasePath = os.path.abspath(os.path.join(os.getenv('RMGpy', '..'), '..', 'RMG-database', 'input'))
logging.info(databasePath)
rmgDatabase.load(databasePath,
                 kineticsFamilies=rxnFamilies,
                 transportLibraries=[],
                 reactionLibraries=[],
                 seedMechanisms=[],
                 thermoLibraries=['primaryThermoLibrary', 'KlippensteinH2O2', 'thermo_DFT_CCSDTF12_BAC', 'CBS_QB3_1dHR' ],
                 solvation=False,
                 )

In [ ]:
import cPickle as pkl
f = open("./autotst/ts_database.pkl", "r")
ts_database = pkl.load(f)
ts_database

In [ ]:
from rmgpy.data.kinetics.family import *
from rmgpy.kinetics import *
from rmgpy.kinetics.arrhenius import ArrheniusEP


In [ ]:
local_context["ArrheniusEP"] = ArrheniusEP
local_context["Arrhenius"] = Arrhenius

In [ ]:
k = KineticsFamily()
k.load("/Users/nathan/Code/RMG-database/input/kinetics/families/H_Abstraction/", local_context, global_context)

In [ ]:
k.generateReactions(r.reactants)
#ts.estimateDistances()

In [ ]:
e1 = ts.groups.top[0]
e1.children

In [ ]:
rmgDatabase.kinetics.families.values()[0]

In [ ]:
c = ts.groups.top[0].item
atomList = c.getPossibleStructures(ts.groups.entries)[0].getLabeledAtoms()
for reactant in b.reactants:
    if isinstance(reactant, Species):
        reactant = reactant.molecule[0]
    # Match labeled atoms
    # Check this reactant has each of the atom labels in this group
    if not all([reactant.containsLabeledAtom(label) for label in atomList]):
        continue # don't try to match this structure - the atoms aren't there!
    # Match structures
    atoms = reactant.getLabeledAtoms()
    
atoms

In [ ]:
global_context = { '__builtins__': None }
local_context={'DistanceData': DistanceData}
path = "../RMG-database/input/kinetics/families/H_Abstraction/"
fpath = os.path.join(path,'TS_training', 'reactions.py')
depository = TransitionStateDepository(label='H_Abstraction/TS_training')
depository.load(fpath, local_context, global_context )

In [ ]:
e = depository.entries['1:H + C3H4 <=> H2 + C3H3']
from rmgpy.data.kinetics.family import KineticsFamily

In [ ]:
TransitionStates().load("../RMG-database/input/kinetics/families/H_Abstraction/", 
                        global_context = { '__builtins__': None },
                        local_context={'DistanceData': DistanceData}
                       )

In [ ]:
k = KineticsFamily()
k.load("/Users/nathan/Code/RMG-database/input/kinetics/families/H_Abstraction/")

In [ ]:
def loadTSDataFile(filePath):
    """
    Load the specified transition state data file and return the dictionary of its contents.

    Returns `None` if the file is invalid or missing.

    Checks that the returned dictionary contains at least rxnLabel, method, qmData.
    """

    if not os.path.exists(filePath):
        return None
    try:
        with open(filePath) as resultFile:
            logging.info('Reading existing ts file {0}'.format(filePath))
            global_context = { '__builtins__': None }
            local_context = {
                '__builtins__': None,
                'True': True,
                'False': False,
                #'QMData': qmdata.QMData,
                'array': numpy.array,
                'int32': numpy.int32,
                'entry': rmgpy.data.base.Entry,
                'DistanceData': DistanceData
                #'distances': dict
            }
            exec resultFile in global_context, local_context
    except IOError, e:
        logging.info("Couldn't read ts file {0}".format(filePath))
        return None
    except (NameError, TypeError, SyntaxError), e:
        logging.error('The ts file "{0}" was invalid:'.format(filePath))
        logging.exception(e)
        return None
    """if not 'rxnLabel' in local_context:
        logging.error('The ts file "{0}" did not contain a rxnLabel.'.format(filePath))
        return None
    if not 'method' in local_context:
        logging.error('The ts file "{0}" did not contain a method.'.format(filePath))
        return None
    if not 'qmData' in local_context:
        logging.error('The ts file "{0}" did not contain thermoData.'.format(filePath))
        return None"""
    return local_context

In [ ]:
loadTSDataFile("../RMG-database/input/kinetics/families/H_Abstraction/TS_groups.py")

In [ ]:
db = Database()

In [ ]:
from qm.transitionstates import DistanceData
f = open("../RMG-database/input/kinetics/families/H_Abstraction/TS_groups.py", "r").read()
local_context = {
            'entry': rmgpy.data.base.Entry,
            #'distances': DistanceData
        }

exec f in local_context

In [ ]:

db.load("../RMG-database/input/kinetics/families/H_Abstraction/TS_groups.py",
        global_context = { '__builtins__': None },
        local_context = {
            'entry': rmgpy.data.base.Entry,
            'distances': DistanceData
        }
       )

In [ ]:
rmgpy.qm.molecule.Geometry?